# Imports

In [23]:
import pandas as pd
import numpy as np

# Manipulation des données

Dans cette partie, je modifie le dataset d'airbnb que j'ai récupéré pour avoir que les données qui me serviront dans le machine learning et que je trouve pertinente.

Pour cela je vais :
1. sélectionner les colonnes pertinentes
2. remplacer les colonnes cleaning_fee et instant_bookable par des 1 et des 0
3. supprimer les appartements qui n'ont pas de reviews (je considère qu'ils n'ont pas été loués)
4. récupérer le prix en $ (actuellement en logarithme)
5. Remplacer les choix contenu dans les colonnes city et cancellation_policy par plusieurs colonnes en 1 et 0

In [29]:
df = pd.read_csv('data/airbnb_data.csv')

df = df[['log_price', 'room_type', 'accommodates', 'cancellation_policy', 'cleaning_fee', 'city', 'first_review', 'instant_bookable', 'number_of_reviews', 'review_scores_rating', 'bedrooms']]
df['cleaning_fee'] = df['cleaning_fee'].astype(int)
df['instant_bookable'] = df['instant_bookable'].apply(lambda x: 1 if x == 'f' else 0)
df = df.dropna(axis=0, subset=['first_review'])
df['price_by_night'] = np.exp(df['log_price'])
df = df.drop(labels='log_price', axis=1)

unique_cities = df['city'].unique()
print(unique_cities) # ['NYC' 'DC' 'SF' 'LA' 'Chicago' 'Boston']
df = pd.get_dummies(df, columns=['city'], dtype=int)

unique_policies = df['cancellation_policy'].unique()
print(unique_policies) # ['strict' 'moderate' 'flexible' 'super_strict_30' 'super_strict_60']
df = pd.get_dummies(df, columns=['cancellation_policy'], dtype=int)

df.head()

['NYC' 'DC' 'SF' 'LA' 'Chicago' 'Boston']
['strict' 'moderate' 'flexible' 'super_strict_30' 'super_strict_60']


,room_type,accommodates,cleaning_fee,first_review,instant_bookable,number_of_reviews,review_scores_rating,bedrooms,price_by_night,city_Boston,city_Chicago,city_DC,city_LA,city_NYC,city_SF,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
0,Entire home/apt,3,1,2016-06-18,1,2,100.0,1.0,150.0,0,0,0,0,1,0,0,0,1,0,0
1,Entire home/apt,7,1,2017-08-05,0,6,93.0,3.0,169.0,0,0,0,0,1,0,0,0,1,0,0
2,Entire home/apt,5,1,2017-04-30,0,10,92.0,1.0,145.0,0,0,0,0,1,0,0,1,0,0,0
4,Entire home/apt,2,1,2015-05-12,0,4,40.0,0.0,115.0,0,0,1,0,0,0,0,1,0,0,0
5,Private room,2,1,2017-08-27,0,3,100.0,1.0,85.0,0,0,0,0,0,1,0,0,1,0,0
